# Part V — APIs with Django  
## 26. API Security and Production Concerns (CORS, Auth, Abuse Prevention, Docs, Key Rotation)

This chapter is about making your API **safe to expose** to real clients (web apps,
mobile apps, third parties) and **safe to operate** in production.

You’ll learn the security and operational decisions professional teams make:

- Which authentication style fits your clients (Session vs Token vs JWT)
- How CSRF changes depending on auth choice
- How to configure CORS correctly (and what *not* to do)
- How to prevent abuse (rate limiting, page-size caps, upload limits)
- How to prevent data leaks (scoping, object permissions, serializer “mass assignment”)
- How to document and version APIs safely (OpenAPI + deprecation policy)
- How to handle secrets, signing keys, and rotation (practical playbook)
- How to log/monitor API behavior without leaking sensitive data

---

## 26.0 Learning Outcomes

By the end of this chapter, you should be able to:

1. Choose an API auth strategy and explain why:
   - session cookies
   - DRF token auth
   - JWT (access + refresh)
2. Configure CSRF correctly for cookie-based API clients (and know when CSRF is not
   relevant).
3. Configure CORS safely for cross-origin frontends.
4. Apply “abuse prevention” controls:
   - throttling
   - pagination caps
   - request body limits
   - upload limits
5. Prevent data leaks via:
   - strict queryset scoping
   - object-level permissions
   - serializer write-field whitelisting
6. Add OpenAPI docs and enforce “API contract” discipline.
7. Implement a basic key rotation strategy for tokens/JWT.
8. Add tests for:
   - authentication
   - permission denial
   - rate limiting (429)
   - CORS headers (when enabled)

---

## 26.1 Threat Model for APIs (What You Must Defend Against)

APIs are attacked differently than HTML pages because they’re easy to automate.

### 26.1.1 Common API threat categories (practical)
- **Broken access control** (most common, highest impact):
  - user reads another user’s org data (tenant leak)
- **Mass assignment**:
  - client sets fields they should not control (created_by, is_staff, role, etc.)
- **Auth attacks**:
  - credential stuffing on token endpoint
  - stolen tokens used from other devices
- **Abuse / DoS**:
  - huge `page_size`, expensive filters, repeated requests
  - large file uploads
- **Injection**:
  - usually mitigated by ORM, but can appear in raw SQL, sorting, filtering if unsafe
- **Data exposure**:
  - returning sensitive fields (emails, internal IDs) unintentionally
- **Misconfiguration**:
  - open CORS, debug mode, allowed hosts, insecure cookies, missing HTTPS

---

## 26.2 Authentication Choices (Session vs Token vs JWT) — Deep, Practical

Your auth choice controls:
- whether CSRF matters
- whether CORS matters
- how clients store credentials
- how you revoke access
- operational complexity

### 26.2.1 Option A — SessionAuthentication (cookie-based)
**Best for:**
- same-origin web apps
- browsable API for developers
- classic Django sites that add JSON endpoints

**How it works:**
- browser stores `sessionid` cookie
- client sends cookie automatically
- server uses session to authenticate

**Security implications:**
- **CSRF is required** for unsafe methods because cookies are sent automatically.
- Token theft via XSS is reduced if session cookie is HttpOnly, but XSS can still
  perform actions as user (because it runs in user’s origin).

**Operational:**
- session storage backend must be reliable (DB/Redis)
- easy logout (flush session)

### 26.2.2 Option B — DRF TokenAuthentication (static token)
**Best for:**
- simple API clients (scripts, integrations)
- internal services
- mobile apps (sometimes) if rotation is handled manually

**How it works:**
- client stores a token string
- client sends:
  - `Authorization: Token <token>`
- server validates token

**Security implications:**
- **CSRF is not required** (token not auto-sent by browser).
- If token is stolen, attacker can use it until revoked.
- Requires careful client storage (don’t put in localStorage if your web app is
  vulnerable to XSS; in web apps prefer HttpOnly cookies or short-lived tokens).

**Operational:**
- easy to implement
- token revocation = delete token row
- no built-in refresh/rotation by default

### 26.2.3 Option C — JWT (access + refresh tokens)
**Best for:**
- SPAs/mobile apps
- multi-service architectures
- when you need short-lived access tokens + refresh flows

**How it works:**
- client sends:
  - `Authorization: Bearer <access_token>`
- access token is short-lived
- refresh token obtains new access token

**Security implications:**
- CSRF not required if tokens are in Authorization header (not cookies).
- Revocation and rotation require more design.
- If you store refresh tokens insecurely, compromise is severe.

**Operational:**
- extra endpoints (token obtain/refresh/verify/revoke)
- key management (signing keys)
- potentially blacklists/rotation policies

### 26.2.4 “Industry default” decision guidance
- If your API is primarily consumed by **your own browser pages on the same domain**:
  - SessionAuthentication is fine and simplest.
- If your API is consumed by **mobile apps or third-party clients**:
  - Token or JWT is more appropriate.
- If your frontend is a **separate domain SPA**:
  - JWT (or cookie-based session with strict CSRF + CORS) is common; choose
    deliberately.

---

## 26.3 CSRF in APIs (When It Matters and When It Doesn’t)

### 26.3.1 CSRF matters when:
- authentication relies on cookies (session, or cookie-stored tokens)
- browser automatically includes credentials

### 26.3.2 CSRF does not apply the same way when:
- client sends `Authorization: Bearer ...` manually
- a malicious site cannot force the browser to include that header automatically

### 26.3.3 The common mistake
People enable cookie-based auth for APIs and disable CSRF checks to “make it work.”
That is a real vulnerability.

Correct choices are:
- keep cookie auth + CSRF properly configured, OR
- move to token/JWT auth for cross-origin API clients

---

## 26.4 CORS (Cross-Origin Resource Sharing) — Correct Configuration

CORS is a **browser** security mechanism. It controls whether JavaScript running on
one origin can read responses from another origin.

### 26.4.1 When you need CORS
You need CORS when:
- your frontend runs on a different origin than the API

Example:
- frontend: `https://app.example.com`
- API: `https://api.example.com`

Browser will block JS from reading API responses unless API returns correct CORS headers.

### 26.4.2 CORS does *not* protect you from CSRF
CORS controls reading responses.
CSRF is about sending requests that change state with cookies attached.

You can be vulnerable to CSRF even if CORS is “tight.”

### 26.4.3 Use `django-cors-headers` (common industry tool)
Install (add to runtime dependencies):

```bash
python -m pip install django-cors-headers
python -m pip freeze > requirements.txt
```

Add to `INSTALLED_APPS`:

```python
INSTALLED_APPS += ["corsheaders"]
```

Add middleware **near the top** (must run before CommonMiddleware in typical setups):

```python
MIDDLEWARE = [
    "corsheaders.middleware.CorsMiddleware",
    "django.middleware.security.SecurityMiddleware",
    # ...
] + MIDDLEWARE
```

### 26.4.4 Safe baseline CORS config (token/JWT auth, no cookies)
In settings:

```python
CORS_ALLOWED_ORIGINS = [
    "https://app.example.com",
]
CORS_ALLOW_CREDENTIALS = False
```

This means:
- JS from `app.example.com` can read responses
- cookies are not included by browsers automatically (good if you don’t use cookie auth)

### 26.4.5 Cookie-based auth + CORS (harder; do intentionally)
If you choose cookie-based auth for a cross-origin SPA, you need:

```python
CORS_ALLOWED_ORIGINS = [
    "https://app.example.com",
]
CORS_ALLOW_CREDENTIALS = True
CSRF_TRUSTED_ORIGINS = [
    "https://app.example.com",
]
```

And your frontend must send credentials:

```javascript
fetch("https://api.example.com/api/v1/...", {
  credentials: "include",
  ...
});
```

**Security notes:**
- Never set `CORS_ALLOW_ALL_ORIGINS=True` with `CORS_ALLOW_CREDENTIALS=True`.
  That combination is extremely dangerous.
- When using cookie auth cross-origin, CSRF must be handled carefully.

### 26.4.6 Preflight (OPTIONS) — what it is and why it matters
For certain requests (e.g., non-simple methods/headers), browsers send an OPTIONS request first:

- method: OPTIONS
- asks: “Is it allowed to send POST with Authorization header?”

Your server must respond with the correct CORS headers. `django-cors-headers` handles this.

---

## 26.5 Preventing Abuse (Rate Limits, Caps, and Limits)

APIs must be resilient against “valid-looking” but abusive traffic.

### 26.5.1 Pagination caps (mandatory)
In DRF, ensure:

- `page_size_query_param` exists only if you cap it (`max_page_size`)
- otherwise attacker can request huge pages and overload DB and memory

You already created:

```python
class StandardResultsPagination(PageNumberPagination):
    page_size_query_param = "page_size"
    max_page_size = 100
```

Keep `max_page_size` and test it.

### 26.5.2 Throttling (DRF-level)
DRF throttling is good for:
- baseline abuse prevention
- protecting endpoints like comments, login, uploads

But for serious production traffic you typically also rate limit at:
- CDN/WAF
- Nginx/Envoy
- API gateway

DRF throttling config example:

```python
REST_FRAMEWORK = {
    # ...
    "DEFAULT_THROTTLE_CLASSES": [
        "rest_framework.throttling.AnonRateThrottle",
        "rest_framework.throttling.UserRateThrottle",
    ],
    "DEFAULT_THROTTLE_RATES": {
        "anon": "60/min",
        "user": "600/min",
        "comments": "10/min",
        "uploads": "30/hour",
    },
}
```

And per-view/action throttling using `ScopedRateThrottle`.

### 26.5.3 Request body size limits (protect memory)
In Django settings:

```python
DATA_UPLOAD_MAX_MEMORY_SIZE = 10 * 1024 * 1024
FILE_UPLOAD_MAX_MEMORY_SIZE = 5 * 1024 * 1024
```

Also consider limiting JSON payload size in DRF parsers (usually handled by Django request limits).

### 26.5.4 File upload limits (protect storage)
Validate in serializer/form:
- size
- type
- image decode sanity (Pillow parsing)

Also consider:
- per-user quotas
- upload rate limits
- virus scanning (advanced, but common in enterprise)

---

## 26.6 Data Leak Prevention (Scoping + Object Permissions + Serializer Whitelists)

### 26.6.1 Scoping: always enforce tenant/org boundary in queryset
Your TaskViewSet got this right:

- org determined from URL and membership
- queryset filtered by org

The “never do this” example:

```python
Task.objects.all()
```

Even if permission checks exist, a bug in permission logic could leak data. Scoping
in the queryset reduces blast radius.

### 26.6.2 Object permissions: enforce at the object boundary
DRF object permission hooks (`has_object_permission`) ensure:
- even if user can hit the endpoint, they can’t modify forbidden objects

### 26.6.3 Serializer mass assignment (very common API bug)
If you use `ModelSerializer` and include fields like:

- `organization`
- `created_by`
- `updated_by`
- `author`
- `is_staff`
- `role`

…then a client may be able to set them unless:
- they’re excluded from `fields`, or
- marked read-only, and you enforce server-side assignment in `perform_create`

**Industry rule:** serializers must be **write-whitelists**, not “dump all fields.”

Example good Task serializer approach:
- accept only editable fields
- set org/created_by/updated_by in server code

### 26.6.4 Don’t expose sensitive fields accidentally
Common sensitive fields:
- user email, last_login, is_superuser
- internal IDs that enable enumeration across orgs
- file system paths (instead of safe URLs)
- secrets/tokens

Use dedicated serializers for public vs private contexts.

---

## 26.7 Production API Documentation (OpenAPI) — “Industry Standard”

Most professional APIs publish OpenAPI docs.

### 26.7.1 Minimal built-in schema (no third-party)
DRF includes a schema view you can start with. It’s not always as rich, but it’s a baseline.

### 26.7.2 Common industry tools (choose one)
- drf-spectacular (widely used)
- drf-yasg (older, still used in many projects)

You want:
- `/api/schema/` (OpenAPI JSON)
- `/api/docs/` (Swagger or Redoc UI)

### 26.7.3 Example setup (drf-spectacular style)
Install:

```bash
python -m pip install drf-spectacular
python -m pip freeze > requirements.txt
```

Settings:

```python
INSTALLED_APPS += ["drf_spectacular"]

REST_FRAMEWORK = {
    # ...
    "DEFAULT_SCHEMA_CLASS": "drf_spectacular.openapi.AutoSchema",
}

SPECTACULAR_SETTINGS = {
    "TITLE": "Django Mastery API",
    "DESCRIPTION": "API documentation",
    "VERSION": "1.0.0",
}
```

URLs:

```python
from django.urls import path
from drf_spectacular.views import SpectacularAPIView, SpectacularSwaggerView

urlpatterns += [
    path("api/schema/", SpectacularAPIView.as_view(), name="api_schema"),
    path("api/docs/", SpectacularSwaggerView.as_view(url_name="api_schema")),
]
```

**Operational rule:** protect docs if needed (internal APIs), or keep public docs
limited to what you’re comfortable exposing.

---

## 26.8 API Versioning Policy (Beyond `/v1/`)

### 26.8.1 What to document
- supported versions (v1, v2)
- how long old versions are supported
- how clients learn about deprecation (headers, docs, announcements)

### 26.8.2 Deprecation pattern
Common:
- Add response header: `Deprecation: true` and/or `Sunset: <date>`
- Provide migration notes

Even if you don’t implement these headers now, write the policy.

---

## 26.9 Token/JWT Key Management and Rotation (Practical Playbook)

### 26.9.1 Secrets you must treat carefully
- Django `SECRET_KEY`
- email credentials
- database passwords
- JWT signing keys (if using JWT)
- token generation keys
- third-party API keys (Stripe, etc.)

### 26.9.2 Rotation goals
- rotate compromised keys quickly
- rotate routinely without breaking users

### 26.9.3 JWT rotation strategy (practical)
Common patterns:
- keep access tokens short-lived (minutes)
- use refresh tokens with rotation (one-time refresh tokens)
- store refresh tokens server-side (blacklist or token family tracking) OR accept stateless risk explicitly

Key rotation patterns:
- maintain multiple verifying keys:
  - new tokens signed with current key
  - old tokens still accepted for a grace period using old key
- after grace, retire old key

Even if you don’t implement multi-key verification now, you should design for it if
you choose JWT.

### 26.9.4 DRF Token auth rotation (simple)
- tokens are stored server-side
- rotation = delete old token and create a new one
- clients must update stored token

This is simpler but less feature-rich.

---

## 26.10 LAB A — Add TokenAuthentication (Simple, Real API Client Support)

If you want your API to be consumable by scripts/mobile without CSRF complexity,
implement token auth (simpler than JWT).

### 26.10.1 Enable authtoken app
In settings:

```python
INSTALLED_APPS += ["rest_framework.authtoken"]
```

Migrate:

```bash
python manage.py migrate
```

### 26.10.2 Configure DRF auth classes
In settings:

```python
REST_FRAMEWORK = {
    # ...
    "DEFAULT_AUTHENTICATION_CLASSES": [
        "rest_framework.authentication.TokenAuthentication",
        # Keep SessionAuthentication if you want browsable API logins too:
        "rest_framework.authentication.SessionAuthentication",
    ],
}
```

### 26.10.3 Add token obtain endpoint
In `config/urls.py`:

```python
from rest_framework.authtoken.views import obtain_auth_token

urlpatterns += [
    path("api/v1/auth/token/", obtain_auth_token, name="api_token_auth"),
]
```

### 26.10.4 Use token from a client
1) Obtain token:

```bash
curl -i -X POST http://127.0.0.1:8000/api/v1/auth/token/ \
  -H "Content-Type: application/json" \
  --data '{"username":"admin","password":"pass12345"}'
```

Response:

```json
{ "token": "abc123..." }
```

2) Call a protected endpoint:

```bash
curl -i http://127.0.0.1:8000/api/v1/orgs/acme/tasks/ \
  -H "Authorization: Token abc123..."
```

### 26.10.5 Production notes
- Add throttling to the token endpoint (credential stuffing defense).
- Consider requiring HTTPS always; tokens over HTTP are easily intercepted.
- Consider token rotation policy (manual logout endpoint or periodic rotation).

---

## 26.11 LAB B — Add JWT Auth (Access + Refresh) (More Production-Common for SPAs)

If you choose JWT, a common approach is `djangorestframework-simplejwt`.

High-level steps (do not skip threat modeling):
- install simplejwt
- configure authentication class
- add token obtain/refresh endpoints
- set access token lifetime short, refresh token longer
- consider refresh token rotation + blacklisting

Because JWT details vary a lot by product, implement only if your client needs it.

---

## 26.12 Idempotency (Prevent Duplicate Creates on Retries) — Practical API Concern

This is essential for:
- payments
- order creation
- invitations

### 26.12.1 Simple idempotency pattern (concept)
- client sends `Idempotency-Key` header on POST
- server stores result keyed by (user, endpoint, idempotency_key)
- retry returns stored result, not a second create

### 26.12.2 Lightweight implementation sketch (advanced)
You can implement an `IdempotencyKey` model storing:
- key
- user
- path
- method
- response status + body

Then in middleware or per-view wrapper:
- if key exists, return stored response
- else process request and store response

This is more advanced; implement when you truly need it (payments/orders).

---

## 26.13 Logging & Monitoring for APIs (Without Leaking Secrets)

### 26.13.1 What to log for API requests
- request_id
- method, path, status
- user_id (if authenticated)
- org_slug (if relevant)
- duration_ms
- throttle hits (429)
- permission denied (403/404 policy)

### 26.13.2 What NOT to log
- Authorization headers (tokens)
- cookies
- raw request bodies (especially auth payloads)

### 26.13.3 Add specific audit logs for sensitive actions
Examples:
- exporting CSV
- changing roles
- publishing content

Use structured logs like:

```python
logger.info(
    "event=tasks_export org=%s actor_id=%s",
    org.slug,
    request.user.id,
)
```

---

## 26.14 Tests for API Security (High-Value Regression Suite)

You want tests that prevent the most expensive failures:

### 26.14.1 Authentication required for writes
- POST/PATCH without auth → 401
- with auth but insufficient permissions → 403
- with auth and allowed → 200/201

### 26.14.2 Tenant scoping
- outsider cannot list org tasks → 404 (your chosen policy)
- outsider cannot retrieve task by ID even if they guess it → 404

### 26.14.3 Throttling
- after N requests → 429

### 26.14.4 CORS (if enabled)
- requests from allowed origin contain `Access-Control-Allow-Origin`
- requests from disallowed origin do not

Example CORS test (conceptual; exact headers depend on your configuration):

```python
from rest_framework.test import APITestCase


class CorsTests(APITestCase):
    def test_cors_allows_configured_origin(self):
        r = self.client.get(
            "/api/v1/articles/",
            HTTP_ORIGIN="https://app.example.com",
        )
        self.assertEqual(r.status_code, 200)
        self.assertEqual(
            r.headers.get("Access-Control-Allow-Origin"),
            "https://app.example.com",
        )
```

---

## 26.15 Production Checklist (API Security and Ops)

### Authentication & Authorization
- [ ] Decide auth strategy (session/token/JWT) and document it
- [ ] Writes require auth; reads limited appropriately
- [ ] Object-level permissions enforced and tested
- [ ] Tenant/org scoping enforced in querysets and tested

### CSRF & CORS
- [ ] If cookie-based auth: CSRF enforced and tested
- [ ] If cross-origin frontend: CORS configured with explicit origins
- [ ] Never use `allow all origins` + `allow credentials`

### Abuse prevention
- [ ] Pagination caps
- [ ] throttling for sensitive endpoints (auth, comments, uploads)
- [ ] request and file size limits
- [ ] input validation for filters/sort/search

### Documentation & versioning
- [ ] OpenAPI schema published (internal/public decision)
- [ ] Versioning strategy (`/api/v1/`) and deprecation policy documented

### Secrets & key rotation
- [ ] secrets in env vars (not code)
- [ ] rotation plan (who, how, when)
- [ ] short-lived access tokens if using JWT

### Observability
- [ ] request IDs
- [ ] access logs without secrets
- [ ] metrics plan (latency/error rate)
- [ ] alerting for spikes (429/5xx)

---

## 26.16 Exercises (Do These Before Proceeding)

1. Implement **TokenAuthentication** and:
   - create an API token for a user
   - call `/api/v1/orgs/<org>/tasks/` with Authorization header
   - write a test that POST without token returns 401

2. Add throttling to the token endpoint:
   - configure `anon: 5/min` for `/api/v1/auth/token/`
   - write a test that eventually returns 429

3. Configure CORS for a fake SPA origin:
   - allow `https://app.example.com`
   - write a test verifying CORS header is present for allowed origin

4. Add an explicit page size cap test:
   - request `page_size=999999`
   - assert response uses capped value (or returns 400, depending on your policy)

5. Write an “API breaking change” example and how you’d introduce it in `/api/v2/`.

---

Next chapter: **Part VI — 27. ASGI, Async Views, and Django’s Async Capabilities**  
We’ll clarify what async in Django actually means, where it helps, where it doesn’t,
how ASGI changes deployment, and how to write safe async endpoints without creating
hidden performance bugs.